Questo script e' stato fatto per analizzare il motivo per cui alcuni non Ponzi (circa il 15% di quelli presenti nel test set) vengono classificati come Ponzi. 

In [1]:
import numpy as np
import pandas as pd
import statistics
from scipy import stats

In [2]:
# leggo le stats dei ponzi del mio dataset
ponzi = pd.read_csv(r'C:\Users\andre\Desktop\materiale tesi\settimana 16\nuovi script etherscan\risultati\ponzi_result.csv', header = 0)
features_names_ponzi = list(ponzi.columns) # ottengo i nomi delle features
fn_without_address_ponzi = features_names_ponzi[:] # copio il nome delle features
fn_without_address_ponzi.pop(0) # rimuovo il primo campo addresses, in quanto non e' una feature

# leggo le stats dei non ponzi del mio dataset
non_ponzi = pd.read_csv(r'C:\Users\andre\Desktop\materiale tesi\settimana 16\nuovi script etherscan\risultati\non_ponzi_result.csv', header = 0)
features_names_non_ponzi = list(non_ponzi.columns) # ottengo i nomi delle features
fn_without_address_non_ponzi = features_names_non_ponzi[:] # copio il nome delle features
fn_without_address_non_ponzi.pop(0) # rimuovo il primo campo addresses, in quanto non e' una feature

# leggo la lista dei falsi positivi: i non ponzi classificati come ponzi
false_positive = pd.read_csv(r'C:\Users\andre\Desktop\materiale tesi\settimana 16\dataset\cleaned+\fbeta\false positive.csv', header = 0)
features_names_false_positive = list(false_positive.columns)
pd.set_option('max_columns', None) # per poter stampare tutte le colonne
pd.set_option('max_rows', None)

In [3]:
ponzi_wo_addr = ponzi.values[:, 1:] # rimuovo la colonna contenente gli indirizzi
# creo un dataframe basandomi su ponzi_wo_addr
df_ponzi = pd.DataFrame(ponzi_wo_addr, columns = fn_without_address_ponzi) 

non_ponzi_wo_addr = non_ponzi.values[:, 1:] # rimuovo la colonna contenente gli indirizzi
# creo un dataframe basandomi su non_ponzi_wo_addr
df_non_ponzi = pd.DataFrame(non_ponzi_wo_addr, columns = fn_without_address_non_ponzi) 

Adesso ho due dataframe con tutte le features sia per i ponzi che per i non ponzi. Eseguendo .describe() sui rispettivi dataframe pandas ottengo informazioni statistiche con cui confrontarli coi falsi positivi

In [4]:
# effettuo le .describe() su ponzi e su non ponzi
ponzi_stats_dataframe = df_ponzi.astype(float).describe(include='all')
non_ponzi_stats_dataframe = df_non_ponzi.astype(float).describe(include = 'all')

In [5]:
# creo un DataFrame dei falsi positivi
fp_wo_addr = false_positive.values[:, 2:]
fp_addresses = false_positive.values[:, 1]
df_fp = pd.DataFrame(fp_wo_addr, columns = fn_without_address_non_ponzi)

# effettuo le .describe() sul DataFrame per ottenere informazioni sui falsi positivi
df_fp_stats = df_fp.astype(float).describe()
df_fp_stats

,balance,lifetime,tx_in,tx_out,investment_in,payment_out,investment_to_contract/tx_in,payment_from_contract/tx_out,#addresses_paying_contract,#addresses_paid_by_contract,mean_v1,sdev_v1,mean_v2,sdev_v2,paid_rate,paid_one,percentage_some_tx_in,sdev_tx_in,percentage_some_tx_out,sdev_tx_out,known_rate,owner_gets_eth_Wo_investing,owner_gets_eth_investing,owner_no_eth
count,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000
mean,305.909469,125.614035,180.508772,126.087719,123.070175,126.008772,0.427936,0.754331,79.596491,82.868421,-0.604145,2.199090,-0.092412,290.783445,0.818111,0.499453,0.743418,19.100449,0.413635,6.015715,0.696098,0.131579,0.315789,0.552632
std,3264.395909,312.768837,870.829604,945.664121,854.746546,945.673179,0.274999,0.425375,704.403215,779.754165,2.654581,4.365979,0.449543,3057.167066,0.785410,0.426872,0.846380,57.547612,0.499041,43.835149,0.441832,0.339525,0.466882,0.499417
min,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-20.000000,0.000000,-4.184702,0.000000,0.000000,0.000000,0.003700,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,4.000000,1.000000,1.000000,1.000000,0.184425,0.814275,1.000000,1.000000,-1.000000,0.000000,-0.005762,0.000000,0.069950,0.000000,0.064175,0.000000,0.003825,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.500000,11.000000,2.000000,3.000000,2.000000,0.484700,1.000000,2.000000,1.000000,0.000000,0.614474,0.000000,0.041000,0.791300,0.500000,0.645850,1.024891,0.094500,0.000000,1.000000,0.000000,0.000000,1.000000
75%,0.013672,96.000000,100.750000,17.250000,22.500000,17.250000,0.600000,1.000000,7.000000,5.000000,0.027778,2.000000,0.000000,0.791010,1.018300,1.000000,1.000000,6.996642,1.000000,1.494149,1.000000,0.000000,1.000000,1.000000
max,34854.380471,1956.000000,8996.000000,9998.000000,8992.000000,9998.000000,0.999600,1.000000,7522.000000,8329.000000,4.000000,24.222293,0.000000,32645.292844,5.000000,1.000000,7.000000,497.239913,2.000000,467.329498,1.000000,1.000000,1.000000,1.000000


Per effettuare un confronto piu' rapido, mantengo solo le 3 features con maggior importanza nella classifica e che dunque pesano di piu' nella classificazione.

In [6]:
df_fp_stats2 = df_fp_stats[['paid_one', 'known_rate', 'tx_in']]
df_fp_stats2

,paid_one,known_rate,tx_in
count,114.000000,114.000000,114.000000
mean,0.499453,0.696098,180.508772
std,0.426872,0.441832,870.829604
min,0.000000,0.000000,2.000000
25%,0.000000,0.000000,4.000000
50%,0.500000,1.000000,11.000000
75%,1.000000,1.000000,100.750000
max,1.000000,1.000000,8996.000000


In [7]:
ponzi_stats_dataframe2 = ponzi_stats_dataframe[['paid_one', 'known_rate', 'tx_in']]
ponzi_stats_dataframe2

,paid_one,known_rate,tx_in
count,673.000000,673.000000,673.000000
mean,0.443041,0.645629,198.156018
std,0.404492,0.450212,716.665369
min,0.000000,0.000000,2.000000
25%,0.000000,0.000000,5.000000
50%,0.482800,1.000000,20.000000
75%,0.848500,1.000000,197.000000
max,1.000000,1.000000,9691.000000


In [8]:
non_ponzi_stats_dataframe2 = non_ponzi_stats_dataframe[['paid_one', 'known_rate', 'tx_in']]
non_ponzi_stats_dataframe2

,paid_one,known_rate,tx_in
count,3884.000000,3884.000000,3884.000000
mean,0.143661,0.248151,2135.774202
std,0.314830,0.414673,3677.714855
min,0.000000,0.000000,1.000000
25%,0.000000,0.000000,5.000000
50%,0.000000,0.000000,43.500000
75%,0.000000,0.500000,2005.250000
max,1.000000,1.000000,19485.000000


Come si puo' notare i valori statistici delle 3 features piu' importanti sono estremamente simili tra falsi positivi e Ponzi. 
Adesso conto il numero di campi uguali che hanno i falsi positivi e i Ponzi: conto per ogni valore presente in un contratto falso positivo, quanti Ponzi hanno lo stesso valore di quella particolare features

In [9]:
i = 0
counter = 0
list_of_lists = []

while i < len(df_fp):
    j = 0
    inner_list = []
    while j < len(df_fp.columns):
        analyze = df_fp.iloc[i, j]
        l = (df_ponzi.iloc[:, j]).tolist()
        counter = l.count(analyze)
        inner_list.append(counter)
        j += 1
    list_of_lists.append(inner_list)
    i += 1
    
df_fin = pd.DataFrame(list_of_lists, index = fp_addresses, columns = fn_without_address_non_ponzi)

Dal risultato si osserva che la maggior parte dei Falsi positivi ha valori delle due Features piu' importanti (paid_one e known_rate), uguali alla maggior parte dei contratti Ponzi.

In [10]:
fp_df_equal_fields = df_fin[['paid_one', 'known_rate', 'tx_in']]
fp_df_equal_fields

,paid_one,known_rate,tx_in
0x615D2c5155eA9841f2a926c3A4953d140D407C99,2,352,7
0x324e5e2335b8e8bdb11ec9ab7db176d7e2a875e1,0,352,2
0xc48ff50311a4c019bfe5f7f552eed87af70172b9,14,352,7
0xdd8a7295d2ea1a9d492631b8e8e3d36ec83123db,246,199,13
0xccf13891822bf2de0cacd300d2ce6f27abce8abe,138,352,29
0x60f0ee673e413e1c134d1497eeecce8ddd2fdd74,246,199,79
0x393de12bd7ae3045068ee9fdbb24a0e61c84b618,246,199,79
0x28297291aebef85bed1b15b9d5fca3146c6ebb3f,246,199,31
0x33A8eA1c8C6294C9F65f3DAd7CA7f037BD09F951,138,352,0
0x50f7da5ecd7838b1f5a4813cdfb59d7d502dce44,246,199,79
